<a href="https://colab.research.google.com/github/itberrios/CV_tracking/blob/main/setup_tutorials/tutorial_yolo_nas_and_ocsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO-NAS + OC-SORT**

This notebook contains a tutorial that shows how to incorporate YOLO-NAS with OC-SORT to perform real time visual tracking on a data stream. In this case, we will use a YouTube video to simulate a data stream of image frames.

## **YOLO-NAS**
[YOLO-NAS](https://github.com/Deci-AI/super-gradients/blob/master/YOLONAS.md) is a powerful object detector with an optimal neural network architecture that has been selected using Neural Architecture Search (NAS), hence the name NAS. 

At the time of release, it outperforms all of the other single shot object detectos in terms of speed and accuracy. It also excels in an area that most single shot detector struggle with, small objects. Two-stage detectors typically perform better than single stage detectors on small objects, at the cost of increased detection time [source](https://arxiv.org/pdf/1907.09408.pdf). YOLO-NAS, howver seems to provide a good tradeoff between detection speed and accuracy on small objects that previous versions of YOLO have not been able to deliver.

## **OC-SORT**
[OC-SORT](https://arxiv.org/abs/2203.14360) is a robust visual obejct tracking algorithm that improves upon the already popular [SORT](https://arxiv.org/abs/1602.00763) algorithm.

SORT tends to loose track on obejcts when they are lost for extended periods of time or when non-linear motion occurs. Algorithms such as Deep SORT have effectoively improved SORT in these scenarios with a Deep Association metric that is computed with a [Siamese Neural Network](https://arxiv.org/pdf/1707.02131.pdf) over the image patches. Eventhough this is effective it comes with the cost of increased detection time due to the deep association and the Siamese network needs to be trained on in-domain data for this approach to be effective. OC-SORT on the otherhand is able to effectively increase tracking performance in a model free fashion with minimal impact to inference speed.

OC-SORT introduces
- Observation Centric Re-Update (ORU)
    - Reduces accumulated Kalman Filter error/uncertainty when a lost track is re-associated
- Observation Centric Momentum (OCM)
    - Uses previous observations to compute a low noise expected motion direction and incorporates it into the track association cost
- Observation Centric Recovery (OCR)
    - Uses the last known observation as a secondary association to help prevent lost tracks

For more details and a break down of each technique that OC-SORT introduces, please see this [article](https://medium.com/@itberrios6/introduction-to-ocsort-c1ea1c6adfa2).

#### Install Libraries

In [1]:
# sketchy fix for "https://stackoverflow.com/questions/73711994/importerror-cannot-import-name-is-directory-from-pil-util-usr-local-lib"
!pip install fastcore -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# supergradients installs
! pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113 &> /dev/null
! pip install super-gradients
! pip install pytorch-quantization==2.1.2 --extra-index-url https://pypi.ngc.nvidia.com &> /dev/null
! pip install matplotlib==3.1.3 &> /dev/null
! pip install --upgrade psutil==5.9.2 &> /dev/null
! pip install --upgrade pillow==7.1.2 &> /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.2/964.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 

In [3]:
!pip install filterpy
!pip install pytube
!pip install moviepy
!pip install ffmpeg

# bug fix for imageio-ffmpeg
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=435b2c5a129aed2ff9639ded59dbb0de2eb0010495110883d8e4e99df18461b6
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.5.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=c44f2f10aecf7eaed32dca5782e7c66097cfaa4476d70e78153aebffeee2d668
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=f925b6ea4ff720997a820176e47e0eebcc1d6325ce89e38a7252abee13962839
  Stored in directory: /root/.cache/pip/wheels/96/5d/ce/bdbdb04744dac03906336eb0d01ff1e222061d3419c55c55f9
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 

### Get OCSORT code

In [4]:
!mkdir ocsort
%cd ocsort
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/ocsort.py 
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/kalmanfilter.py 
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/association.py 

%cd ..

/content/ocsort
--2023-05-09 18:16:31--  https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/ocsort.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18109 (18K) [text/plain]
Saving to: ‘ocsort.py’

ocsort.py           100%[===================>]  17.68K  --.-KB/s    in 0.001s  

2023-05-09 18:16:31 (27.2 MB/s) - ‘ocsort.py’ saved [18109/18109]

--2023-05-09 18:16:31--  https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/kalmanfilter.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 

### Import Libraries

In [1]:
import os
import time
from tqdm import tqdm

import numpy as np
import cv2
import filterpy

import torch
import super_gradients as sg
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)

The console stream is logged into /root/sg_logs/console.log


[2023-05-09 18:18:51] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-09 18:18:59] WARNING - env_sanity_check.py - Failed to verify installed packages: matplotlib==3.1.3 does not satisfy requirement matplotlib>=3.3.4


## Download video from YouTube

In [2]:
from pytube import YouTube

# url =r"https://www.youtube.com/watch?v=wIYD42DV3Ro" # horse racing
url = r"https://www.youtube.com/watch?v=JteKbauGolo" # nascar
yt = YouTube(url)
print("Video Title: ", yt.title)

# download video
video_path = yt.streams \
  .filter(progressive=True, file_extension='mp4') \
  .order_by('resolution') \
  .desc() \
  .first() \
  .download() 

Video Title:  Chastain BANZAI last lap move!!


## Instantiate YOLO-NAS model

In [59]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get("yolo_nas_s", pretrained_weights="coco").cuda()
model.eval();

## Check Model Inference Speed

#### First get a test frame

In [61]:
cap = cv2.VideoCapture(video_path)
if (cap.isOpened() == False):
    print("Error opening video file")

while(cap.isOpened()):

  # read each video frame
  ret, frame = cap.read()
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

  break

cap.release()
del cap

In [76]:
# baseline speed test to get detections for tracker
image_in = frame.copy()
base_test_times = []

for i in range(100):
  # start clock and rpedict
  tic = time.perf_counter()
  preds_0 = model.predict(image_in)
  img_preds = list(preds_0._images_prediction_lst)[0]
  dets = np.hstack((img_preds.prediction.bboxes_xyxy, 
                    np.c_[img_preds.prediction.confidence]))
  base_test_times.append(time.perf_counter() - tic)

print(f"Average Prediction time: {np.mean(base_test_times)} "
      f"Average FPS: {1/np.mean(base_test_times)}")

In [77]:
np.round(np.mean(base_test_times), 4), np.round(1/np.mean(base_test_times), 4)

(0.0637, 15.6903)

Get new method of inference with reduced overhead. We can explore places in the API to leverage by finding where the predcit function is located. We can locate it with the inspect library.





In [66]:
import inspect

os.path.abspath(inspect.getfile(model.predict))

'/usr/local/lib/python3.10/dist-packages/super_gradients/training/models/detection_models/customizable_detector.py'

In [88]:
from super_gradients.training.models.detection_models.customizable_detector import CustomizableDetector
from super_gradients.training.pipelines.pipelines import DetectionPipeline

# make sure to set IOU and confidence in the pipeline constructor
pipeline = DetectionPipeline(
            model=model,
            image_processor=model._image_processor,
            post_prediction_callback=model.get_post_prediction_callback(iou=0.25, conf=0.30),
            class_names=model._class_names,
        )


def get_prediction(image_in, pipeline):
  ''' Obtains DetectionPrediction object from a single input RGB image
  '''
  # Preprocess
  preprocessed_image, processing_metadata = pipeline.image_processor.preprocess_image(image=image_in.copy())

  # Predict
  with torch.no_grad():
      torch_input = torch.Tensor(preprocessed_image).unsqueeze(0).to('cuda')
      model_output = model(torch_input)
      # output = (model_output[2].unsqueeze(0), model_output[0])
      prediction = pipeline._decode_model_output(model_output, model_input=torch_input)

  # Postprocess
  return pipeline.image_processor.postprocess_predictions(predictions=prediction[0], metadata=processing_metadata)

In [83]:
image_in = frame.copy()
test_times = []

for i in range(100):
  # start clock and rpedict
  tic = time.perf_counter()
  pred = get_prediction(image_in, pipeline)
  xyxyc = np.hstack((pred.bboxes_xyxy, 
                     np.c_[pred.confidence]))
  test_times.append(time.perf_counter() - tic)

print(f"Average Prediction time: {np.mean(test_times)} "
      f"Average FPS: {1/np.mean(test_times)}")

In [84]:
np.round(np.mean(test_times), 4), np.round(1/np.mean(test_times), 4)

(0.0495, 20.1928)

In [75]:
prediction

DetectionPrediction(bboxes_xyxy=array([[6.54378479e+02, 4.30875610e+02, 7.27754517e+02, 4.89263977e+02],
       [1.01892291e+03, 6.20825073e+02, 1.16830432e+03, 7.20033752e+02],
       [3.66328003e+02, 4.53606720e+02, 4.48491211e+02, 5.36878540e+02],
       [7.18938782e+02, 3.89818726e+02, 8.09202026e+02, 4.53293060e+02],
       [6.61631653e+02, 3.83896881e+02, 7.53329224e+02, 4.54482330e+02],
       [2.87815063e+02, 4.35830292e+02, 3.36478485e+02, 4.99735443e+02],
       [8.48887712e-02, 3.96302521e+02, 1.26962036e+02, 4.76873383e+02],
       [3.65796478e+02, 4.52874573e+02, 4.48642059e+02, 5.35653076e+02],
       [6.78801025e+02, 2.16158340e+02, 7.38538757e+02, 2.33446045e+02],
       [2.55704620e+02, 3.03850616e+02, 2.88912933e+02, 3.19262054e+02],
       [7.89146301e+02, 4.81799286e+02, 8.03686035e+02, 5.16868652e+02],
       [2.86529083e+02, 4.34711060e+02, 3.35552429e+02, 4.98740753e+02],
       [5.59906555e+02, 3.07066406e+02, 1.27999329e+03, 3.82853790e+02],
       [4.24226288e

In [72]:
img_preds.prediction

DetectionPrediction(bboxes_xyxy=array([[6.54378479e+02, 4.30875610e+02, 7.27754517e+02, 4.89263977e+02],
       [1.01892291e+03, 6.20825073e+02, 1.16830432e+03, 7.20033752e+02],
       [3.66328003e+02, 4.53606720e+02, 4.48491211e+02, 5.36878540e+02],
       [7.18938782e+02, 3.89818726e+02, 8.09202026e+02, 4.53293060e+02],
       [6.56734009e+02, 3.82992096e+02, 8.10075500e+02, 4.88658508e+02],
       [6.61631653e+02, 3.83896881e+02, 7.53329224e+02, 4.54482330e+02],
       [2.87815063e+02, 4.35830292e+02, 3.36478485e+02, 4.99735443e+02],
       [7.07427795e+02, 3.89608398e+02, 8.12091614e+02, 4.82169342e+02],
       [8.48887712e-02, 3.96302521e+02, 1.26962036e+02, 4.76873383e+02],
       [3.65796478e+02, 4.52874573e+02, 4.48642059e+02, 5.35653076e+02],
       [6.78801025e+02, 2.16158340e+02, 7.38538757e+02, 2.33446045e+02],
       [2.55704620e+02, 3.03850616e+02, 2.88912933e+02, 3.19262054e+02],
       [7.89146301e+02, 4.81799286e+02, 8.03686035e+02, 5.16868652e+02],
       [2.86529083e

## Perform inference on Simulated Video Stream

#### First we will use MoviePy to get the frame rate and save the audio for later

In [85]:
from moviepy.editor import VideoFileClip

videoclip = VideoFileClip(video_path)
audioclip = videoclip.audio

video_fps = videoclip.fps
video_fps

29.97002997002997

## Instantiate tracker object

In [86]:
from ocsort import ocsort

tracker = ocsort.OCSort(det_thresh=0.25)

Helper function for bounding box colors

In [87]:
import colorsys    

def get_color(number):
    """ Converts an integer number to a color """
    # change these however you want to
    hue = number*30 % 180
    saturation = number*103 % 256
    value = number*50 % 256

    # expects normalized values
    color = colorsys.hsv_to_rgb(hue/179, saturation/255, value/255)

    return [int(c*255) for c in color]

## Now we can simualate the data stream using opencv

Make sure to reset the tracker each time you run the inference

In [99]:
# get frame info for tracker and video saving 
h, w = (720, 1280)
h2, w2 = h//2, w//2
# h2, w2 = 640, 640 # this degrades performance
img_size = (h2, w2) 
img_info = (h, w)

In [113]:
# ensure tracker is reset
tracker = ocsort.OCSort(det_thresh=0.30, max_age=10, min_hits=2)

cap = cv2.VideoCapture(video_path)

if (cap.isOpened() == False):
    print("Error opening video file")

frames = []
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while(cap.isOpened()):

  # read each video frame (read time is about 0.006 sec)
  ret, frame = cap.read()

  if ret == True:

    # read image and resize by half for inference
    og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(og_frame, 
                       (w2, h2), interpolation=cv2.INTER_LINEAR)

    # perform inference on small frame and get (x1, y1, x2, y2, confidence)
    pred = get_prediction(frame, pipeline)
    xyxyc = np.hstack((pred.bboxes_xyxy, 
                      np.c_[pred.confidence]))

    # update tracker
    tracks = tracker.update(xyxyc, img_info, img_size)

    # draw tracks on frame
    for track in tracker.trackers:
      
      track_id = track.id
      hits = track.hits
      color = get_color(track_id*15)
      x1,y1,x2,y2 = np.round(track.get_state()).astype(int).squeeze()

      cv2.rectangle(og_frame, (x1,y1),(x2,y2), color, 2)
      cv2.putText(og_frame, 
                  f"{track_id}-{hits}", 
                  (x1+10,y1-5), 
                  cv2.FONT_HERSHEY_SIMPLEX, 
                  0.5,
                  color, 
                  1,
                  cv2.LINE_AA)
      
    # update FPS and place on frame
    current_time = time.perf_counter()
    elapsed = (current_time - start_time)
    counter += 1
    if elapsed > 1:
      fps = counter / elapsed;
      counter = 0;
      start_time = current_time;


    cv2.putText(og_frame, 
                f"FPS: {np.round(fps, 2)}", 
                (10,h - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,
                (255,255,255), 
                2,
                cv2.LINE_AA)

    # append to list
    frames.append(og_frame)

    # TEMP for debug
    if i == 10:
      break
    else:
      i += 1

  # Break the loop
  else:
    break

# release video capture object
cap.release()
del cap

In [110]:
fps, counter, elapsed, counter/elapsed

(0, 11, 0.6750942789994951, 16.29402046822593)

In [112]:
plt.imshow(frames[4])

## Now let's put this in a video

In [337]:
# save to mp4
out = cv2.VideoWriter(video_savepath,
                      cv2.VideoWriter_fourcc(*'MP4V'), 
                      video_fps,
                      (w, h))
 
for frame in frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

out.release()
del out

### Now add back the audio

In [339]:
from moviepy.editor import CompositeAudioClip
detection_video = VideoFileClip(video_savepath)

# add sound and save
detection_video.audio = CompositeAudioClip([audioclip])
detection_video.write_videofile('chastain_wall_ride_with_audio.mp4')

Moviepy - Building video chastain_wall_ride_with_audio.mp4.
MoviePy - Writing audio in chastain_wall_ride_with_audioTEMP_MPY_wvf_snd.mp3


t:   0%|          | 3/1049 [00:00<00:35, 29.35it/s, now=None]

MoviePy - Done.
Moviepy - Writing video chastain_wall_ride_with_audio.mp4

